# FRED data

Documentation can be found in this [link](https://fred.stlouisfed.org/docs/api/fred/). But we are going to use the package `fredapi` to get the data

In [2]:
# uncoment if you need to install
#!pip install fredapi

In [1]:
import pandas as pd
import fredapi as fa
import os

# path for the folder "project"
path = "C:\\Users\\pedro\\OneDrive\\NYU\\CSS\\II. Data Skills\\project"
os.chdir(path)

import keys # file with api keys
key = keys.FRED
fred = fa.Fred(api_key = key)

In [2]:
# function
def fred_api(series_id, series_name, unit, seasonal):
    """
    Get series from fred and add metadata information to it.
    
    ---
    Parameters:
        series_id [str]: series individual id from fred
        series_name [str]: How would you like to call the series
        unit [str]: type of data (YoY, MoM, Index, Dollar Value...)
        seasonal [str]: seasonal adjust (S) or without adjust (U)
    
    ---
    Returns: Pandas DataFrame with series and details
    
    """
    
    series = fred.get_series(series_id).reset_index().dropna()
    series = series.rename(columns = {series.columns[0]:"date", series.columns[1]:"value"})
    series["series_id"] = series_id
    series["series_name"] = series_name
    series["unit"] = unit
    series["seasonal"] = seasonal
    
    return series[["series_id","date","value","series_name","unit","seasonal"]]

## Housing

- `CSUSHPINSA`: S&P/Case-Shiller U.S. National Home Price Index (Jan2000=100, Not Seasonally Adjusted)
- `CSUSHPISA`: S&P/Case-Shiller U.S. National Home Price Index (Jan2000=100, Seasonally Adjusted)
- `MORTGAGE30US`: 30-Year Fixed Rate Mortgage Average in the United States

### Case Shiller

In [4]:
# CASE SHILLER (NSA)
csnsa = fred_api(series_id = "CSUSHPINSA",
                 series_name = "CS Home Price Index",
                 unit = "Index", seasonal = "U")

# CASE SHILLER (SA)
cssa = fred_api(series_id = "CSUSHPISA",
                 series_name = "CS Home Price Index",
                 unit = "Index", seasonal = "S")

# concat
case_shiller = pd.concat([csnsa, cssa])

case_shiller.info()
case_shiller.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 858 entries, 144 to 572
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   series_id    858 non-null    object        
 1   date         858 non-null    datetime64[ns]
 2   value        858 non-null    float64       
 3   series_name  858 non-null    object        
 4   unit         858 non-null    object        
 5   seasonal     858 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 46.9+ KB


,series_id,date,value,series_name,unit,seasonal
144,CSUSHPINSA,1987-01-01,63.734,CS Home Price Index,Index,U
145,CSUSHPINSA,1987-02-01,64.134,CS Home Price Index,Index,U
146,CSUSHPINSA,1987-03-01,64.470,CS Home Price Index,Index,U
147,CSUSHPINSA,1987-04-01,64.974,CS Home Price Index,Index,U
148,CSUSHPINSA,1987-05-01,65.548,CS Home Price Index,Index,U


In [5]:
case_shiller.to_parquet("data\\housing\\data_fred_caseshiller.parquet")

### Mortgage Rate

In [6]:
# Mortgage Rate
mortgage = fred_api(series_id = "MORTGAGE30US",
                    series_name = "30 Years Mortgage Rate",
                    unit = "Percent", seasonal = "U")

mortgage.info()
mortgage.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2696 entries, 0 to 2695
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   series_id    2696 non-null   object        
 1   date         2696 non-null   datetime64[ns]
 2   value        2696 non-null   float64       
 3   series_name  2696 non-null   object        
 4   unit         2696 non-null   object        
 5   seasonal     2696 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 126.5+ KB


,series_id,date,value,series_name,unit,seasonal
0,MORTGAGE30US,1971-04-02,7.33,30 Years Mortgage Rate,Percent,U
1,MORTGAGE30US,1971-04-09,7.31,30 Years Mortgage Rate,Percent,U
2,MORTGAGE30US,1971-04-16,7.31,30 Years Mortgage Rate,Percent,U
3,MORTGAGE30US,1971-04-23,7.31,30 Years Mortgage Rate,Percent,U
4,MORTGAGE30US,1971-04-30,7.29,30 Years Mortgage Rate,Percent,U


In [7]:
mortgage.to_parquet("data\\housing\\data_fred_mortgage.parquet")

## Inflation

- `GASREGW`: US Regular All Formulations Gas Price
- `PFOODINDEXM`: Global Food Price Index

### Gasoline Prices

In [8]:
gas = fred_api(series_id = "GASREGW",
                 series_name = "US Weekly Gas Price",
                 unit = "Dollars per Gallon", seasonal = "U")
gas.info()
gas.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1679 entries, 0 to 1684
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   series_id    1679 non-null   object        
 1   date         1679 non-null   datetime64[ns]
 2   value        1679 non-null   float64       
 3   series_name  1679 non-null   object        
 4   unit         1679 non-null   object        
 5   seasonal     1679 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 91.8+ KB


,series_id,date,value,series_name,unit,seasonal
0,GASREGW,1990-08-20,1.191,US Weekly Gas Price,Dollars per Gallon,U
1,GASREGW,1990-08-27,1.245,US Weekly Gas Price,Dollars per Gallon,U
2,GASREGW,1990-09-03,1.242,US Weekly Gas Price,Dollars per Gallon,U
3,GASREGW,1990-09-10,1.252,US Weekly Gas Price,Dollars per Gallon,U
4,GASREGW,1990-09-17,1.266,US Weekly Gas Price,Dollars per Gallon,U


In [9]:
gas.to_parquet("data\\inflation\\data_fred_gasprice.parquet")

In [7]:
# Global food index
food = fred_api(series_id = "PFOODINDEXM",
                   series_name = "Global Food Price Index",
                   unit = "Index", seasonal = "U")

food.to_parquet("data\\inflation\\data_fred_foodprice.parquet")

## Employment

- `ICSA`: Initial Jobless Claims SA
- `CCSA`: Continued Claims SA

In [3]:
# Jobless Claims
iclaims = fred_api(series_id = "ICSA",
                   series_name = "US Initial Jobless Claims",
                   unit = "Number", seasonal = "S")

cclaims = fred_api(series_id = "CCSA",
                   series_name = "US Contined Claims",
                   unit = "Number", seasonal = "S")

claims = pd.concat([iclaims,cclaims])
claims.head()

,series_id,date,value,series_name,unit,seasonal
0,ICSA,1967-01-07,208000.0,US Initial Jobless Claims,Number,S
1,ICSA,1967-01-14,207000.0,US Initial Jobless Claims,Number,S
2,ICSA,1967-01-21,217000.0,US Initial Jobless Claims,Number,S
3,ICSA,1967-01-28,204000.0,US Initial Jobless Claims,Number,S
4,ICSA,1967-02-04,216000.0,US Initial Jobless Claims,Number,S


In [5]:
claims.to_parquet("data\\employment\\data_fred_jobless_claims.parquet")

## Monetary Policy and Financial Market

- `FEDFUNDS`: Fed Funds  
- `DGS1`: 1 Year Treasury Rate
- `DGS2`: 2 Years Treasury Rate
- `DGS5`: 5 Years Treasury Rate
- `DGS10`: 10 Year Treasury Rate
- `DGS30`: 30 Year Treasury Rate
- `T5YIE`: 5 Years Breakeven Rate
- `T10YIE`: 10 Years Breakeven Rate
- `T5YIFR`: 5 Years, 5 Years Forward Inflation Expectation Rate
- `RESPPANWW`: Fed Total Assets (Millions of U.S. Dollars)
- `BOGZ1FL192090005Q`: Households Net Worth, Level (Millions of Dollars)
- `DTWEXBGS`: Nominal Broad U.S. Dollar Index
- `M2SL`: M2 Seasonally Adjusted


### Rates

In [3]:
# Fed funds
ff = fred_api(series_id = "RIFSPFFNB",
              series_name = "FED Funds Rate",
                   unit = "Rate", seasonal = "U")

ff.to_parquet("data\\monetary_market\\data_fred_fed_funds.parquet")

# treasury yields
ust1 = fred_api(series_id = "DGS1",
                   series_name = "US 1 Year Treasury Rate",
                   unit = "Rate", seasonal = "U")

ust2 = fred_api(series_id = "DGS2",
                   series_name = "US 2 Years Treasury Rate",
                   unit = "Rate", seasonal = "U")

ust5 = fred_api(series_id = "DGS5",
                   series_name = "US 5 Years Treasury Rate",
                   unit = "Rate", seasonal = "U")

ust10 = fred_api(series_id = "DGS10",
                   series_name = "US 10 Years Treasury Rate",
                   unit = "Rate", seasonal = "U")

ust30 = fred_api(series_id = "DGS30",
                   series_name = "US 30 Years Treasury Rate",
                   unit = "Rate", seasonal = "U")

ust = pd.concat([ust1,ust2,ust5,ust10,ust30])

ust.to_parquet("data\\monetary_market\\data_fred_treasury_yields.parquet")

### Breakevens

In [4]:
be5 = fred_api(series_id = "T5YIE",
                   series_name = "US 5 Years Breakeven Rate",
                   unit = "Rate", seasonal = "U")

be10 = fred_api(series_id = "T10YIE",
                   series_name = "US 10 Years Breakeven Rate",
                   unit = "Rate", seasonal = "U")

be55 = fred_api(series_id = "T5YIFR",
                   series_name = "US 5Y-5Y Forward Inflation Expectation",
                   unit = "Rate", seasonal = "U")

be = pd.concat([be5,be10,be55])

be.to_parquet("data\\monetary_market\\data_fred_breakeven_rates.parquet")

### Others

In [3]:
# Fed Total Assets
fedassets = fred_api(series_id = "RESPPANWW",
                     series_name = "FED Total Assets",
                     unit = "Millions of Dollars", seasonal = "U")

fedassets.to_parquet("data\\monetary_market\\data_fred_fed_assets.parquet")

# Households Net Worth, Level (Millions of Dollars)
networth = fred_api(series_id = "BOGZ1FL192090005Q",
                     series_name = "Households Net Worth",
                     unit = "Millions of Dollars", seasonal = "U")

networth.to_parquet("data\\monetary_market\\data_fred_households_networth.parquet")

# Nominal Broad U.S. Dollar Index
usd = fred_api(series_id = "DTWEXBGS",
                     series_name = "Nominal Broad U.S. Dollar Index",
                     unit = "Index", seasonal = "U")

usd.to_parquet("data\\fiscal_trade\\data_fred_usd.parquet")

#M2 
m2 = fred_api(series_id = "M2SL",
                     series_name = "M2",
                     unit = "Billions of Dollars", seasonal = "S")

m2.to_parquet("data\\monetary_market\\data_fred_m2.parquet")

## Fiscal

- `MTSO133FMS`: Total Federal Outlays (Millions of Dollars, Not Seasonally Adjusted)
- `MTSR133FMS`: Total Federal Receipts (Millions of Dollars, Not Seasonally Adjusted)

In [8]:
outlays = fred_api(series_id = "MTSO133FMS",
                   series_name = "Total Federal Outlays",
                   unit = "Millions of Dollars", seasonal = "U")

receipts = fred_api(series_id = "MTSR133FMS",
                   series_name = "Total Federal Receipts",
                   unit = "Millions of Dollars", seasonal = "U")

government = pd.concat([outlays,receipts])

government.to_parquet("data\\fiscal_trade\\data_fred_government_accounts.parquet")